<img src="https://cdn.comet.ml/img/notebook_logo.png">

[Hugging Face](https://huggingface.co/docs) is a community and data science platform that provides tools that enable users to build, train and deploy ML models based on open source (OS) code and technologies. Primarily known for their `transformers` library, Hugging Face has helped democratized access to these models by providing a unified API to train and evaluate a number of popular models for NLP.

Comet integrates with Hugging Face's `Trainer` object, allowing you to log your model parameters, metrics, and assets such as model checkpoints. Learn more about our integration [here](https://www.comet.com/docs/v2/integrations/ml-frameworks/huggingface/)

Curious about how Comet can help you build better models, faster? Find out more about [Comet](https://www.comet.com/site/products/ml-experiment-tracking/?utm_campaign=transformers&utm_medium=colab) and our [other integrations](https://www.comet.ml/docs/v2/integrations/overview/)


Get a preview for what's to come. Check out a completed experiment created from this notebook [here](https://www.comet.com/examples/comet-examples-transformers-trainer/3992ddee441f446bbb65c3cc4c8bd33b)

# Install Comet and Dependencies

In [1]:
%pip install comet_ml torch datasets transformers scikit-learn accelerate


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: /home/lothiraldan/.virtualenvs/comet3.9/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Initialize Comet

In [2]:
import comet_ml

comet_ml.init(project_name="comet-examples-transfomers-trainer")

# Set Model Type

In [3]:
PRE_TRAINED_MODEL_NAME = "distilbert/distilroberta-base"
SEED = 42

# Load Data

In [4]:
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

2024-05-29 11:36:21.081337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 11:36:21.081382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 11:36:21.083078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 11:36:21.090918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 11:36:22.317986: W tensorflow/compiler/tf2

# Setup Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [6]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
    )


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create Sample Datasets

For this guide, we are only going to sample 200 examples from our dataset.  

In [8]:
train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(200))
eval_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(200))

# Setup Transformer Model

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, num_labels=2
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Setup Evaluation Function

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def get_example(index):
    return eval_dataset[index]["text"]


def compute_metrics(pred):
    experiment = comet_ml.get_running_experiment()

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)

    if experiment:
        epoch = int(experiment.curr_epoch) if experiment.curr_epoch is not None else 0
        experiment.set_epoch(epoch)
        experiment.log_confusion_matrix(
            y_true=labels,
            y_predicted=preds,
            file_name=f"confusion-matrix-epoch-{epoch}.json",
            labels=["negative", "postive"],
            index_to_example_function=get_example,
        )

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Run Training

In order to enable logging from the Hugging Face Trainer, you will need to set the `COMET_MODE` environment variable to `ONLINE`.  If you would like to log assets produced in the training run as Comet Assets, set `COMET_LOG_ASSETS=TRUE`   

In [11]:
%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

training_args = TrainingArguments(
    seed=SEED,
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_total_limit=10,
    save_steps=25,
    per_device_train_batch_size=8,
    report_to=["comet_ml"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)
trainer.train()

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


/home/lothiraldan/project/cometml/transformers/src/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lothiraldan/comet-examples-transfomers-trainer/5308c4cebfff4a9db8b425b6515bbb55



Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.628739,0.595000,0.494997,0.781081,0.578125


TrainOutput(global_step=25, training_loss=0.6734719085693359, metrics={'train_runtime': 425.5504, 'train_samples_per_second': 0.47, 'train_steps_per_second': 0.059, 'total_flos': 26493479731200.0, 'train_loss': 0.6734719085693359, 'epoch': 1.0})

In [12]:
comet_ml.get_running_experiment().end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : registered_ampere_8049
COMET INFO:     url                   : https://www.comet.com/lothiraldan/comet-examples-transfomers-trainer/5308c4cebfff4a9db8b425b6515bbb55
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch                    : 1.0
COMET INFO:     eval_accuracy            : 0.595
COMET INFO:     eval_f1                  : 0.4949967268306369
COMET INFO:     eval_loss                : 0.6287391781806946
COMET INFO:     eval_precision           : 0.7810810810810811
COMET INFO:     eval_recall              : 0.578125
COMET INFO:     eval_runtime             : 98.7791
COMET INFO:     eval_samples_per_second  : 2.025
COMET INFO:     e